<a href="https://colab.research.google.com/github/misha-solntcev/-/blob/master/Brat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pandas
! pip install natasha

In [11]:
import pandas as pd
df = pd.read_excel('/content/Братищенко/ЗаявкиТест.xlsx')

In [19]:
with open('/content/Братищенко/StopListRus.txt', 'r') as file:
    stoplist = list(file)

stoplist = [x[:-1] for x in stoplist]

In [20]:
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

def text2lemmas(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return ' '.join([token.lemma for token in doc.tokens])


In [23]:
# Лемматизация
df['lemmas'] = [text2lemmas(item) for item in df['КРАТКОЕ_ОПИСАНИЕ']]


In [28]:
# Векторайзер
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(stop_words=stoplist, token_pattern='[А-Яа-я]\\w+')

In [30]:
# Мешок слов
bow = count_vectorizer.fit_transform(df['lemmas'])

In [32]:
# Словарь корпуса
v = count_vectorizer.vocabulary_

In [39]:
# Матрица частот корпуса
M = bow.todense()
# print(M)

# вычисление абсолютных частот по матрице
f_term = bow.sum(axis=0)
# print(f_term)

# получение индексов сумм, отсортированных по убыванию
term_i = f_term.argsort()[0,::-1]
# term_i

# создание словаря индекс: слово
i_term = dict(zip(count_vectorizer.vocabulary_.values(), count_vectorizer.vocabulary_.keys()))
# i_term

# список слов по убыванию частот
word_f = [[i_term[term_i[0,j]], f_term[0,term_i[0,j]]] for j in range(term_i.shape[1])]
# word_f

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]
